<a href="https://colab.research.google.com/github/LucyKinyua/Week2_MS/blob/main/Moringa_Data_Science_Prep_W2_Independent_Project_2021_05_Lucy_Kinyua_SQL_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview

In this part of the assessment, you will act as a Data analyst to answer a research question on the US elections. 

First, will be required to write a Data Report on the process that will undertake while working on the given research problem. Using the CRISP-DM Methodology,  you will document the various stages of the data science lifecycle for the given research problem while also providing your recommendation.

You have been provided with a detailed description of what you will be expected to do below.

Do remember that complex tasks that may seem hard at first can often be broken down into a sequence of simple tasks, and there are workarounds to do what first seems “impossible” with a succession of known operations.

**Problem Description**

You have been asked to help a candidate to become US president. The winner of the election will be the candidate winning the most grand electors. 

Grand electors are attributed at the state level: in each of the 51 states, there is a given number of grand electors to win (roughly, but not exactly, proportional to the size of the state) and the presidential candidate receiving the most local votes wins ALL the Grand Electors in that state.

Because the number of grand electors is not exactly proportional to the population, some states can be prioritized to increase the return on investment of the campaign. We assume here there are only 2 candidates, and no history (no trend of certain states to vote for a particular candidate or party). Hence, each vote is equally "expensive" to get, but some states grant more grand elector per capita.

You are provided with 2 tables: one giving the number of Grand Electors per state, the other the population per state.

- Grand Electors by State

- Population by State


You are asked to identify the states that should be prioritized to win the election, with a smart but simple algorithm (brute force computation of all possibilities to find the absolute optimum is not accepted, as it would be to computationally expensive). It is ok not to find the overall optimum, we just want a strategy that makes sense.

(This is of course a very simplistic view of reality, but it is a nice starting point to play with data and analyze possibilities).

First take a few minutes to think about what you need to do. Steps will be suggested hereafter, but a big part of the data scientist's job is to identify the flow of operations when being asked a practical question, so it is important you start exercising on that in addition to programming.

Here is what we are suggesting to do: we will rank states by decreasing number of grand electors per capita. The first states in the list will be the most valuable (you get a large number of grand electors by convincing a small number of people to vote for you). We will target all the states at the top of the list until the cumulative sum (also called running total) of grand electors won is larger than half the total number of Grand Electors in the country.

**Instructions**

To do that, we need (you are allowed to create as many intermediary tables as you want, to keep queries short):

1. To join the 2 tables:
- You notice States are not capitalized the same way in both tables (one is in uppercase letters, the other not), so you will first need to convert all to uppercase, for instance.
- Now you can join the tables on the state key.
2. Your boss wants you to change the name of the "District of Columbia" state to its short version "DC". Please do that.
3. To compute the ratio between the number of grand electors and the population. Please create a new column with that ratio.
4. To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
5. To compute the running total of Grand Electors in that sorted list.
- Hint: you can get inspiration from here to compute a running total from here:  https://stackoverflow.com/questions/21382766/cumulative-summing-values-in-sqlite
6. Independently, to compute the half of the total of Grand Electors overall (in the whole country):
- This is the threshold we need to reach for winning the presidential election.
7. To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold. (the other states can be ignored). That is our target list.
- Hint: You can do that in 2 steps:
 - Select all the states for which the running total is below or equal to the threshold.
 - Add the first state for which the running total is larger than the threshold.

Can you draw some conclusions from the result? Is it in line with your expectations? How many states do you end up with in the target list? Is it a small or a large number? Do you think it would be a good recommendation to target those states?

# Connecting to database and importing data from CSV files

In [1]:
# Loading an sql extension to allow me to work with sql on Colaboratory;
# 
%load_ext sql

# Connecting to the in memory sqlite database;
# 
%sql sqlite://

'Connected: @None'

In [2]:
# Importing the python csv library to allow me to read csv file(s) that  will have uploaded to this environment;
# 
import csv

# Importing the pandas library to use for data manipulation in this notebook;
# 
import pandas as pd

In [3]:
# Uploading the Grand Electors csv file;
# 
with open('GrandElectors_by_state.csv','r') as f:
    GrandElectors_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')

%sql PERSIST GrandElectors_by_state;

 * sqlite://


'Persisted grandelectors_by_state'

In [4]:
# Previewing the Grand_Electors table;
# 
%%sql
SELECT * FROM GrandElectors_by_state
LIMIT 10;

 * sqlite://
Done.


State,GrandElectors
Alabama,9
Alaska,3
Arizona,11
Arkansas,6
California,55
Colorado,9
Connecticut,7
District of Columbia,3
Delaware,3
Florida,29


In [5]:
# Uploading the Population csv file;
# 
with open('Population_by_state.csv','r') as f:
    Population_by_state = pd.read_csv(f, index_col=0, encoding='utf-8')

%sql PERSIST Population_by_state;

 * sqlite://


'Persisted population_by_state'

In [6]:
# Previewing the Population table;
# 
%%sql
SELECT * FROM Population_by_state
LIMIT 10;

 * sqlite://
Done.


State,Population
CALIFORNIA,39144818
TEXAS,27469114
FLORIDA,20271272
NEW YORK,19795791
ILLINOIS,12859995
PENNSYLVANIA,12802503
OHIO,11613423
GEORGIA,10214860
NORTH CAROLINA,10042802
MICHIGAN,9922576


# Data Preparation

In [7]:
# Instruction 1:
# Notice States are not capitalized the same way in both tables
# Converting States in GrandElectors_by_state table to UPPERCASE;
# 
%%sql
UPDATE GrandElectors_by_state SET State = upper(State);

SELECT * FROM GrandElectors_by_state;

 * sqlite://
51 rows affected.
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9
CONNECTICUT,7
DISTRICT OF COLUMBIA,3
DELAWARE,3
FLORIDA,29


In [8]:
# Previewing the Grand_Electors table to confirm the update has taken effect;
# 
%%sql
SELECT * FROM GrandElectors_by_state LIMIT 10;

 * sqlite://
Done.


State,GrandElectors
ALABAMA,9
ALASKA,3
ARIZONA,11
ARKANSAS,6
CALIFORNIA,55
COLORADO,9
CONNECTICUT,7
DISTRICT OF COLUMBIA,3
DELAWARE,3
FLORIDA,29


In [9]:
# Counting the number of States in the Grand_Electors table;
# 
%%sql
SELECT COUNT(State) FROM GrandElectors_by_state;

 * sqlite://
Done.


COUNT(State)
51


In [10]:
# Counting the number of States in the Population table;
# 
%%sql
SELECT COUNT(State) FROM Population_by_state;

 * sqlite://
Done.


COUNT(State)
56


In [11]:
# Notice that the number of States are not the same
# This will be corrected after using the INNER JOIN function
# The records from table one and table two would both be returned,...
# ... but only if the values in column one of table one match the values in column one of table two.
# Any records that do not have matching values would not be returned by an INNER JOIN.


In [12]:
# Joining both tables;
# 
%%sql
CREATE TABLE if not exists new_table AS SELECT STATE, POPULATION, GRAND_ELECTORS FROM
(SELECT
GrandElectors_by_state.State AS STATE,
Population_by_state.Population AS POPULATION,
GrandElectors_by_state.GrandElectors AS GRAND_ELECTORS
FROM GrandElectors_by_state
INNER JOIN Population_by_state ON GrandElectors_by_state.State = Population_by_state.State);

SELECT * FROM new_table;

 * sqlite://
Done.
Done.


STATE,POPULATION,GRAND_ELECTORS
ALABAMA,4858979,9
ALASKA,738432,3
ARIZONA,6828065,11
ARKANSAS,2978204,6
CALIFORNIA,39144818,55
COLORADO,5456574,9
CONNECTICUT,3590886,7
DISTRICT OF COLUMBIA,672228,3
DELAWARE,945934,3
FLORIDA,20271272,29


In [13]:
# Previewing the new joint table to confirm the update has taken effect;
# 
%%sql
SELECT * FROM new_table;

 * sqlite://
Done.


STATE,POPULATION,GRAND_ELECTORS
ALABAMA,4858979,9
ALASKA,738432,3
ARIZONA,6828065,11
ARKANSAS,2978204,6
CALIFORNIA,39144818,55
COLORADO,5456574,9
CONNECTICUT,3590886,7
DISTRICT OF COLUMBIA,672228,3
DELAWARE,945934,3
FLORIDA,20271272,29


# Modelling

In [14]:
# Instruction 2:
# Changing the name of the "District of Columbia" State to its short version "DC".
# 
%%sql
UPDATE new_table
SET STATE = "DC"
WHERE STATE = "DISTRICT OF COLUMBIA";

SELECT * FROM new_table;

 * sqlite://
1 rows affected.
Done.


STATE,POPULATION,GRAND_ELECTORS
ALABAMA,4858979,9
ALASKA,738432,3
ARIZONA,6828065,11
ARKANSAS,2978204,6
CALIFORNIA,39144818,55
COLORADO,5456574,9
CONNECTICUT,3590886,7
DC,672228,3
DELAWARE,945934,3
FLORIDA,20271272,29


In [15]:
# Instruction 3:
# To compute the ratio between the number of grand electors and the population.
# Create a new column with that ratio.
# 
# Instruction 4:
# To order the states by decreasing ratio of Grand Electors per capita. That will make our priority list.
# 
%%sql
SELECT STATE, POPULATION, GRAND_ELECTORS,
(POPULATION/GRAND_ELECTORS) AS "POPULATION FOR EVERY 1 GRAND ELECTOR"
FROM new_table
ORDER BY "POPULATION FOR EVERY 1 GRAND ELECTOR" DESC;

 * sqlite://
Done.


STATE,POPULATION,GRAND_ELECTORS,POPULATION FOR EVERY 1 GRAND ELECTOR
TEXAS,27469114,38,722871
CALIFORNIA,39144818,55,711723
FLORIDA,20271272,29,699009
NEW YORK,19795791,29,682613
NORTH CAROLINA,10042802,15,669520
OHIO,11613423,18,645190
VIRGINIA,8382993,13,644845
ILLINOIS,12859995,20,642999
PENNSYLVANIA,12802503,20,640125
NEW JERSEY,8958013,14,639858


In [17]:
# Instruction 5:
# To compute the running total of Grand Electors in that sorted list.
# 
%%sql


In [16]:
%%sql
SELECT STATE, POPULATION, GRAND_ELECTORS, "POPULATION FOR EVERY 1 GRAND ELECTOR",
  SUM(GRAND_ELECTORS) 
  OVER (PARTITION BY STATE ORDER BY "POPULATION FOR EVERY 1 GRAND ELECTOR" DESC)
  AS "POPULATION FOR EVERY 1 GRAND ELECTOR"
FROM new_table;

 * sqlite://
(sqlite3.OperationalError) near "(": syntax error
[SQL: SELECT STATE, POPULATION, GRAND_ELECTORS, "POPULATION FOR EVERY 1 GRAND ELECTOR",
  SUM(GRAND_ELECTORS) 
  OVER (PARTITION BY STATE ORDER BY "POPULATION FOR EVERY 1 GRAND ELECTOR" DESC)
  AS "POPULATION FOR EVERY 1 GRAND ELECTOR"
FROM new_table;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [21]:
%%sql
SELECT
    STATE,
    POPULATION,
    GRAND_ELECTORS,
    "POPULATION FOR EVERY 1 GRAND ELECTOR"
SUM (GRAND_ELECTORS) OVER (ORDER BY "POPULATION FOR EVERY 1 GRAND ELECTOR") AS RUNNING_TOTAL
FROM new_table;

 * sqlite://
(sqlite3.OperationalError) near "(": syntax error
[SQL: SELECT
    STATE,
    POPULATION,
    GRAND_ELECTORS,
    "POPULATION FOR EVERY 1 GRAND ELECTOR"
SUM (GRAND_ELECTORS) OVER (ORDER BY "POPULATION FOR EVERY 1 GRAND ELECTOR") AS RUNNING_TOTAL
FROM new_table;]
(Background on this error at: http://sqlalche.me/e/14/e3q8)


In [ ]:
%%sql
SELECT country, registration_date,registred_users,
  SUM(registred_users) 
  OVER (PARTITION BY country ORDER BY registration_date)
  AS total_users
FROM registration;

SELECT registration_date,registred_users,
  SUM(registred_users) OVER (ORDER BY registration_date)
  AS total_users
FROM registration;

In [ ]:
SELECT
    t.Date,
    Sum(r.KeyColumn1),
    Sum(r.KeyColumn2),
    Sum(r.KeyColumn3)
FROM (SELECT DISTINCT Date FROM MyTable) as t
Left Join MyTable as r On (r.Date < t.Date)
Group By t.Date;

In [16]:
%%sql 
SELECT SUM(GRAND_ELECTORS)
FROM new_table;

 * sqlite://
Done.


SUM(GRAND_ELECTORS)
538


In [ ]:
# Instruction 6:
# Compute half of the total of Grand Electors overall (in the whole country):
# - This is the threshold we need to reach for winning the presidential election.
# 
%%sql
SELECT SUM(GRAND_ELECTORS)

In [ ]:
# Instruction 7:
# To filter our sorted list of states in order to keep only the (top) ones enabling us to reach the computed threshold.
# (the other states can be ignored). That is our target list.
# Hint: You can do that in 2 steps:
# - Select all the states for which the running total is below or equal to the threshold.
# - Add the first state for which the running total is larger than the threshold.
# 
%%sql
SELECT
FROM new_table;

# Evaluation

In [ ]:
# Test against success criteria
# 


# Deployment

In [ ]:
# Conclusion:
# Can you draw some conclusions from the result?
# Is it in line with your expectations?
# How many states do you end up with in the target list?
# Is it a small or a large number?
# Do you think it would be a good recommendation to target those states?
# 
